In [42]:
import numpy as np

def onemax(v):
    return np.sum(v)

def onemax_m(M):
    return np.sum(M,axis=1)

rng = np.random.default_rng()

class UMDA:
    def __init__(self, length, pop_size, tournament_size, max_generations, fitness_function):
        self.length = length
        self.pop_size = pop_size  
        self.max_generations = max_generations  
        self.prob_vector = np.full(length,0.5)
        self.fitness_function = fitness_function
        self.tournament_size=tournament_size

    def generate_population(self):
        #each row is an individual
        return rng.uniform(0,1,(self.pop_size, self.length)) < self.prob_vector
    
    def tournament_selection(self,population,fitness):

        tournament_indices = rng.choice(self.pop_size, size=(self.pop_size, self.tournament_size), replace=True)
        tournament_fitness = fitness[tournament_indices]
        # print(tournament_fitness.shape)
        # print(tournament_indices.shape)
        # print(np.argmax(tournament_fitness, axis=1).shape)
        winner_indices=np.argmax(tournament_fitness, axis=1)
        #winner_indices = tournament_indices[np.argmax(tournament_fitness, axis=1).reshape(-1),np.arange(self.length).reshape(-1)]
        return population[winner_indices]
    
    def score_selection(self,population,fitness):
        div=np.sum(fitness-np.min(fitness))
        if div==0:
            fitness=np.ones(self.pop_size)/self.pop_size
        else:
            fitness=(fitness-np.min(fitness))/div
        chosen_indices=rng.choice(self.pop_size,self.pop_size,p=fitness,replace=True)
        return population[chosen_indices]
    
    def model_estimation(self,population):
        self.prob_vector=np.sum(population,axis=0)/self.pop_size
    
    def learn(self):
        for generation in range(self.max_generations):
            pop=self.generate_population()
            fitness=self.fitness_function(pop)
            #selected=self.tournament_selection(pop,fitness)
            selected=self.score_selection(pop,fitness)
            print(self.prob_vector)
            self.model_estimation(selected)
            print(f'gen {generation}, best individual: {np.max(fitness)}')

        return self.prob_vector,pop

class PBIL:
    def __init__(self, length, pop_size, learning_rate, mutation_prob, mutation_shift, max_generations, fitness_function):
        self.length = length
        self.pop_size = pop_size  
        self.learning_rate = learning_rate  
        self.mutation_prob = mutation_prob  
        self.mutation_shift = mutation_shift  
        self.max_generations = max_generations  
        self.prob_vector = np.full(length,0.5)
        self.fitness_function = fitness_function
        #self.fitness_function = np.vectorize(fitness_function)
    
    def generate_population(self):
        return rng.uniform(0,1,(self.pop_size, self.length)) < self.prob_vector
    
    def update_probability_vector(self, best_individual):
        self.prob_vector = (1 - self.learning_rate) * self.prob_vector + self.learning_rate * best_individual

    def mutate_probability_vector(self):
        mutation_mask = rng.uniform(0,1,self.length) < self.mutation_prob
        random_mutation = rng.uniform(0,1,self.length) < 0.5
        self.prob_vector = (1 - self.mutation_shift) * self.prob_vector * mutation_mask + self.mutation_shift * random_mutation * mutation_mask + self.prob_vector * (1 - mutation_mask)
    
    def learn(self):
        for generation in range(self.max_generations):
            population = self.generate_population().astype(int)
            fitness_values = np.array([self.fitness_function(ind) for ind in population])
            #fitness_values=self.fitness_function(population)

            best_index = np.argmax(fitness_values)
            best_individual = population[best_index]
            best_fitness = fitness_values[best_index]

            self.update_probability_vector(best_individual)
            self.mutate_probability_vector()

            print(f"Generation {generation}: Best Fitness = {best_fitness}")

        return self.prob_vector, best_fitness

In [15]:
n_bits = 20
pop_size = 100
learning_rate = 0.01
mutation_prob = 0.05
mutation_shift = 0.01
max_generations = 1000

pbil = PBIL(n_bits, pop_size, learning_rate, mutation_prob, mutation_shift, max_generations,onemax)
final_prob_vector, best_fitness = pbil.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

Generation 0: Best Fitness = 16
Generation 1: Best Fitness = 17
Generation 2: Best Fitness = 18
Generation 3: Best Fitness = 15
Generation 4: Best Fitness = 16
Generation 5: Best Fitness = 14
Generation 6: Best Fitness = 15
Generation 7: Best Fitness = 16
Generation 8: Best Fitness = 15
Generation 9: Best Fitness = 15
Generation 10: Best Fitness = 16
Generation 11: Best Fitness = 16
Generation 12: Best Fitness = 17
Generation 13: Best Fitness = 17
Generation 14: Best Fitness = 16
Generation 15: Best Fitness = 17
Generation 16: Best Fitness = 16
Generation 17: Best Fitness = 15
Generation 18: Best Fitness = 16
Generation 19: Best Fitness = 18
Generation 20: Best Fitness = 15
Generation 21: Best Fitness = 16
Generation 22: Best Fitness = 16
Generation 23: Best Fitness = 17
Generation 24: Best Fitness = 16
Generation 25: Best Fitness = 17
Generation 26: Best Fitness = 17
Generation 27: Best Fitness = 16
Generation 28: Best Fitness = 18
Generation 29: Best Fitness = 17
Generation 30: Best 

In [31]:
tournament_size=20
pop_size = 100000
max_generations=200
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]
gen 0, best individual: 19
[0.49716 0.60597 0.5518  0.60349 0.44787 0.44574 0.49796 0.57493 0.6937
 0.40021 0.50476 0.51702 0.50525 0.38543 0.46665 0.47815 0.52946 0.49517
 0.46048 0.47554]
gen 1, best individual: 19
[0.58106 0.54924 0.6066  0.782   0.5031  0.43297 0.47821 0.73778 0.81645
 0.39253 0.52364 0.61925 0.45833 0.212   0.48271 0.42001 0.35887 0.58075
 0.54375 0.67695]
gen 2, best individual: 20
[0.55404 0.49731 0.58264 0.72365 0.5525  0.5625  0.44395 0.68984 0.83151
 0.31831 0.48168 0.69823 0.49842 0.29587 0.41015 0.59135 0.28142 0.6959
 0.61968 0.74031]
gen 3, best individual: 19
[0.67336 0.58429 0.58567 0.58263 0.52911 0.73177 0.44776 0.61027 0.84595
 0.42595 0.4601  0.63699 0.58171 0.32916 0.5237  0.54396 0.23063 0.89059
 0.68151 0.84669]
gen 4, best individual: 20
[0.70993 0.56038 0.7319  0.56337 0.43559 0.70011 0.42942 0.59865 0.77041
 0.39113 0.37404 0.54162 0.63064 0.23015 0.47514 0.7378

In [32]:
tournament_size=20
pop_size = 100000
max_generations=200
n_bits=10
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
gen 0, best individual: 10
[0.54098 0.65072 0.58272 0.90066 0.36005 0.45915 0.22257 0.5367  0.29917
 0.49119]
gen 1, best individual: 10
[0.57258 0.48552 0.63639 1.      0.31938 0.36171 0.25787 0.54396 0.38318
 0.63812]
gen 2, best individual: 10
[0.53083 0.56139 0.72649 1.      0.26968 0.11351 0.26517 0.70237 0.26316
 0.40918]
gen 3, best individual: 10
[0.60459 0.56023 0.64467 1.      0.28674 0.19479 0.2686  0.86055 0.33088
 0.41613]
gen 4, best individual: 10
[0.58708 0.45027 0.45847 1.      0.36887 0.03842 0.24967 0.84483 0.229
 0.59379]
gen 5, best individual: 10
[0.60311 0.35651 0.39673 1.      0.24185 0.08731 0.21237 0.92162 0.18647
 0.66837]
gen 6, best individual: 10
[0.5983  0.35342 0.24547 1.      0.08105 0.23525 0.20706 0.81686 0.21616
 0.88746]
gen 7, best individual: 9
[0.66079 0.37045 0.33462 1.      0.02746 0.23473 0.24436 0.87583 0.14929
 0.94802]
gen 8, best individual: 10
[0.65868 0.31607 0.36434 1.      0.      0.22537 0.261

In [36]:
tournament_size=1000
pop_size = 100000
max_generations=200
n_bits=10
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
gen 0, best individual: 10
[0.48869 0.52285 0.52009 0.48451 0.50104 0.49764 0.48829 0.48582 0.50591
 0.51416]
gen 1, best individual: 10
[0.51487 0.55215 0.55957 0.51322 0.51638 0.49252 0.46328 0.49702 0.52148
 0.52079]
gen 2, best individual: 10
[0.55295 0.54104 0.55995 0.519   0.51276 0.49876 0.46105 0.45486 0.47871
 0.50554]
gen 3, best individual: 10
[0.56772 0.52326 0.55971 0.57063 0.47744 0.51929 0.43354 0.46807 0.4918
 0.50464]
gen 4, best individual: 10
[0.57617 0.50359 0.55939 0.56308 0.49916 0.48589 0.42278 0.47117 0.51476
 0.51817]
gen 5, best individual: 10
[0.56367 0.50211 0.58718 0.59358 0.51493 0.50052 0.4085  0.49024 0.50734
 0.4889 ]
gen 6, best individual: 10
[0.62233 0.4936  0.60291 0.57707 0.52549 0.50962 0.45329 0.49729 0.4848
 0.4432 ]
gen 7, best individual: 10
[0.64704 0.48999 0.64236 0.58114 0.53993 0.53256 0.44303 0.53306 0.47419
 0.46833]
gen 8, best individual: 10
[0.6051  0.51055 0.64102 0.55941 0.53574 0.55108 0.43

In [37]:
tournament_size=1000
pop_size = 100000
max_generations=1000
n_bits=10
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
gen 0, best individual: 10
[0.52724 0.53155 0.50437 0.4945  0.52741 0.49674 0.47492 0.53259 0.50902
 0.49315]
gen 1, best individual: 10
[0.53396 0.5561  0.50882 0.43107 0.50807 0.49434 0.47377 0.56344 0.55614
 0.48663]
gen 2, best individual: 10
[0.51192 0.55217 0.50249 0.40678 0.52683 0.50178 0.47925 0.57158 0.55141
 0.49459]
gen 3, best individual: 10
[0.53479 0.53305 0.50797 0.41908 0.51902 0.48185 0.4782  0.57838 0.54727
 0.50445]
gen 4, best individual: 10
[0.51991 0.53004 0.47885 0.40588 0.55849 0.46809 0.47438 0.54349 0.58074
 0.50153]
gen 5, best individual: 10
[0.51467 0.56427 0.46834 0.39305 0.55712 0.46773 0.47063 0.56189 0.57357
 0.50934]
gen 6, best individual: 10
[0.50703 0.57409 0.48431 0.41985 0.55706 0.529   0.42797 0.56744 0.55454
 0.5087 ]
gen 7, best individual: 10
[0.50127 0.59443 0.48515 0.40395 0.5491  0.5274  0.43822 0.58847 0.55016
 0.51602]
gen 8, best individual: 10
[0.52219 0.57875 0.48684 0.40278 0.55923 0.56594 0.

In [44]:
tournament_size=0
pop_size = 100000
max_generations=100
n_bits=20
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]
gen 0, best individual: 19
[0.52919 0.52469 0.52934 0.52747 0.52995 0.52527 0.52604 0.52844 0.52279
 0.52516 0.53035 0.52889 0.53215 0.52919 0.53232 0.5241  0.52805 0.52842
 0.52867 0.53041]
gen 1, best individual: 19
[0.56185 0.55633 0.55435 0.55759 0.55886 0.55614 0.5587  0.55696 0.55092
 0.55634 0.55981 0.55882 0.55816 0.5532  0.56107 0.55391 0.55819 0.55912
 0.56139 0.56116]
gen 2, best individual: 19
[0.58882 0.58505 0.58193 0.58591 0.58533 0.5875  0.58213 0.58084 0.58025
 0.58367 0.58954 0.58204 0.58435 0.58208 0.58626 0.57891 0.58731 0.58693
 0.58915 0.58787]
gen 3, best individual: 20
[0.61433 0.61215 0.6083  0.60861 0.6144  0.61587 0.60823 0.60938 0.60558
 0.61372 0.61054 0.60679 0.61014 0.60736 0.61633 0.60293 0.61367 0.61185
 0.61121 0.61143]
gen 4, best individual: 20
[0.63699 0.63793 0.63316 0.63422 0.63827 0.64052 0.63    0.63439 0.63408
 0.63657 0.6361  0.6295  0.63585 0.63126 0.64073 0.62

In [47]:
def deceptive_onemax_m(M):
    return np.maximum(M.shape[1]-np.max(M,axis=1)*M.shape[1],np.sum(M,axis=1)-1)

tournament_size=0
pop_size = 100000
max_generations=100
n_bits=10
umda = UMDA(n_bits, pop_size, tournament_size, max_generations,deceptive_onemax_m)
final_prob_vector, best_fitness = umda.learn()

print("Final Probability Vector:", final_prob_vector)
print("Best Fitness Achieved:", best_fitness)

[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
gen 0, best individual: 10
[0.56446 0.56246 0.55752 0.56019 0.566   0.55666 0.56113 0.5599  0.563
 0.55881]
gen 1, best individual: 10
[0.61949 0.61475 0.61462 0.61188 0.62155 0.61029 0.61455 0.61314 0.6147
 0.61312]
gen 2, best individual: 10
[0.66825 0.65996 0.66132 0.65922 0.66653 0.65901 0.66107 0.65862 0.66116
 0.65873]
gen 3, best individual: 10
[0.70585 0.69797 0.7028  0.69511 0.70624 0.69731 0.70328 0.69937 0.7006
 0.69938]
gen 4, best individual: 9
[0.73929 0.73699 0.73516 0.7294  0.73838 0.73373 0.73879 0.73503 0.73942
 0.73467]
gen 5, best individual: 9
[0.76925 0.76598 0.7649  0.75697 0.77149 0.76139 0.7689  0.76307 0.77187
 0.76888]
gen 6, best individual: 9
[0.79524 0.79328 0.7918  0.78358 0.80151 0.79236 0.79603 0.79185 0.798
 0.79561]
gen 7, best individual: 9
[0.8217  0.82193 0.81514 0.81268 0.82893 0.81519 0.82429 0.82071 0.82755
 0.82076]
gen 8, best individual: 9
[0.83785 0.83921 0.83595 0.83252 0.84949 0.83427 0.84441 0.841

In [17]:
#numpy sanitychekc
import numpy as np
m=np.ones((10,5))
print(m.shape)
print(m[0].shape)
print(m.shape[0])
print(m[:,0].shape)
print(m)
print(np.sum(m,axis=0))
print(np.sum(m,axis=1))
h=np.ones(3)*2
print(h)
h[m.astype(int)]

(10, 5)
(5,)
10
(10,)
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[10. 10. 10. 10. 10.]
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
[2. 2. 2.]


array([[2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.]])

In [51]:
import numpy as np

def deceptive_onemax(v):
    if np.max(v)==0:
        return len(v)
    return np.sum(v)-1

def cga_onemax(n, func, population_size=100, generations=1000, learning_rate=1/100):
    """
    Compact Genetic Algorithm (CGA) for the OneMax problem.
    
    :param n: Length of the binary string.
    :param population_size: The virtual population size.
    :param generations: Number of generations to run.
    :param learning_rate: Update rate of probability vector.
    :return: Best found solution.
    """
    # Initialize probability vector (each bit has a 0.5 probability of being 1)
    p = np.full(n, 0.5)
    
    for _ in range(generations):
        # Sample two individuals from probability vector
        individual1 = (np.random.rand(n) < p).astype(int)
        individual2 = (np.random.rand(n) < p).astype(int)
        
        # Evaluate fitness
        fitness1 = func(individual1)
        fitness2 = func(individual2)
        
        # Determine winner and loser
        if fitness1 > fitness2:
            winner, loser = individual1, individual2
        else:
            winner, loser = individual2, individual1
        
        # Update probability vector based on the difference
        p += learning_rate * (winner - loser)
        p = np.clip(p, 0, 1)  # Ensure probabilities remain in [0,1]
    
    # Generate final solution from probability vector
    final_solution = (p > 0.5).astype(int)
    return final_solution, func(final_solution)

# Example usage
n = 50  # Length of the binary string
solution, fitness = cga_onemax(n,onemax)
print("Best solution found:", solution)
print("Fitness:", fitness)


n = 10  # Length of the binary string
solution, fitness = cga_onemax(n,deceptive_onemax)
print("Best solution found:", solution)
print("Fitness:", fitness)


Best solution found: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
Fitness: 50
Best solution found: [1 1 1 1 1 1 1 1 1 1]
Fitness: 9
